In [1]:
import os
from dotenv import load_dotenv

load_dotenv("./credentials.env")

True

In [2]:
# Chat model
from langchain.chat_models import ChatOpenAI

def get_llm():
    return ChatOpenAI(
        model_name=os.getenv("OPENAI_MODEL_NAME"),
        openai_api_base=os.getenv("OPENAI_API_BASE"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0,
        model_kwargs={
            "deployment_id": os.getenv("OPENAI_DEPLOYMENT_ID"),
            "api_type": os.getenv("OPENAI_API_TYPE"),
            "api_version": os.getenv("OPENAI_API_VERSION"),
        },
    )

In [3]:
from langchain.document_loaders import YoutubeLoader

def load_documents(url):
    return YoutubeLoader.from_youtube_url(
        url,
        add_video_info=True,
        language=['en', 'ja']
    ).load()

In [4]:
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain

def summarize(docs, llm):
    prompt_template = """Write a concise Japanese summary of the following transcript of Youtube Video.
        ============
            
        {text}
        
        ============
        
        日本語で 500 文字以内で要約した結果は以下の通りです。
    """
    
    with get_openai_callback() as cb:
        chain = load_summarize_chain( 
            llm,
            chain_type="stuff",
            verbose=False,
            prompt=PromptTemplate(template=prompt_template, input_variables=["text"])
        )
        response = chain({"input_documents": docs}, return_only_outputs=True)
    return response['output_text'], cb.total_cost

In [5]:
url = "https://www.youtube.com/watch?v=FZhbJZEgKQ4"

In [6]:
llm = get_llm()
docs = load_documents(url=url)
output, cost = summarize(docs, llm)

In [7]:
output, cost

('このYouTubeビデオの要約は次の通りです。マイクロソフトはAIを活用して、人々の仕事や生活を効率化し、革新をもたらすことを目指しています。マイクロソフトのCopilotは、自然言語処理を活用して、開発者やビジネスパーソンが作業を効率化するための支援を行います。Copilotは、様々な業界や役割において活用され、個別のニーズに合わせてカスタマイズすることができます。また、CopilotはAIと量子コンピューティングの融合により、科学的な発見や革新を加速することも可能です。マイクロソフトは、Copilotを通じて、すべての人々と組織がより効果的に働き、成長することを目指しています。',
 0.031767000000000004)

In [8]:
# https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

assert docs[0].page_content is not None
transcript_text = docs[0].page_content

def split_docs(text:str)-> [Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", "", "。", "!", "！", "?", "？"],
        chunk_size = 15000,
        chunk_overlap = 20,
        length_function = len,
        is_separator_regex = False,
    )
    
    texts = text_splitter.split_text(transcript_text)
    print(f"splitted to {len(texts)} chunks")
    return [Document(page_content=t) for t in texts]

In [9]:
def summarize_documents(docs:[Document]):
    template = """Write a concise Japanese summary of the following transcript of Youtube Video.
        ============
            
        {text}
        
        ============
        
        日本語で 500 文字以内で要約した結果は以下の通りです。
    """
    prompt_template = PromptTemplate(template=template, input_variables=["text"])

    with get_openai_callback() as cb:
        chain = load_summarize_chain(
            llm=llm,
            chain_type="map_reduce",
            map_prompt=prompt_template,
            combine_prompt=prompt_template,
            collapse_prompt=prompt_template,
            verbose=False,
        )

        output = chain(
            inputs=docs,
            return_only_outputs=True,
        )["output_text"]
    return output, cb.total_cost

In [10]:
len(docs)

1

In [11]:
assert docs[0].page_content is not None
text = docs[0].page_content
print(f"text length {len(text)}")

text length 48532


In [12]:
_docs = split_docs(text)
output, total_cost = summarize_documents(_docs)
print(f"{output}, {total_cost}")

splitted to 4 chunks


このYouTubeビデオのトランスクリプトでは、MicrosoftがAIの進化によって新たな時代が訪れることを伝えています。MicrosoftはAIのインフラストラクチャやモデル、データ、ツールチェーンなどで100以上のアップデートを発表し、OpenAIとのパートナーシップを通じて最新のAIモデルを提供しています。また、Microsoftはオープンソースにも取り組んでおり、Azureに「モデルとしてのサービス」オファリングを追加し、NVIDIAとのパートナーシップを拡大しています。さらに、Microsoft Fabricはデータの取り扱い方を再設計し、Azure Quantum Elementsを使用して化学物質を生成する方法を説明しています。量子要素を使用することで、持続可能な化学物質や薬品、先進材料、バッテリーのための新しい分子を設計することができます。最終的には、Copilotを使用して、8億人の人々がパーソナライズされたチューターや医師、メンターにアクセスできるようにすることが目標です。AIは障害を持つ人々の生活にポジティブな影響を与える可能性があります。, 0.06287100000000001
